### Install Library

In [1]:
pip install huggingface_hub==0.23.4 langchain==0.2.5 langchain_community==0.2.5 torch==2.3.0+cu121 transformers==4.40.1 pypdf==4.2.0

  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
  Using cached langchain-0.2.5-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_community-0.2.5-py3-none-any.whl.metadata (2.5 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.3.0+cu121 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch==2.3.0+cu121


In [ ]:
pip install sentence-transformers==3.0.1 chromadb==0.5.3

In [ ]:
!pip install -U langchain-openai langchain_community pypdf

In [3]:
!pip install -q --upgrade transformers huggingface_hub


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
# 파일 및 정규표현식 관련 모듈
import os  # 운영 체제와 상호작용하기 위한 모듈
import re  # 정규 표현식을 사용하기 위한 모듈

# PyTorch 관련 모듈
import torch  # PyTorch 프레임워크

# Transformers 관련 모듈
from transformers import (
    AutoModelForCausalLM,  # 언어 모델을 로드하는 함수
    AutoTokenizer,  # 토크나이저를 로드하는 함수
    BitsAndBytesConfig,  # 양자화 및 압축을 위한 설정
    TrainingArguments,  # 모델 훈련을 위한 인자 설정
    pipeline,  # 파이프라인을 생성하는 함수
    logging,  # 로깅을 위한 함수
    AutoConfig,  # 설정을 로드하는 함수
    GenerationConfig  # 텍스트 생성을 위한 설정
)

# Langchain 관련 모듈
from langchain_community.document_loaders import PyPDFLoader  # PDF 문서 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain.embeddings import HuggingFaceEmbeddings  # 임베딩 생성기
from langchain.schema import Document  # 문서 스키마
from langchain_community.vectorstores import Chroma  # 벡터 저장소
from langchain.chains.question_answering import load_qa_chain  # 질의 응답 체인 로드
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain  # 질의 응답 체인
from langchain.memory import ConversationSummaryMemory  # Langchain 메모리 관련 모듈대화 요약 메모리
from langchain.text_splitter import CharacterTextSplitter  # Langchain 텍스트 분할기 모듈 문자 기반 텍스트 분할기
from langchain.prompts import PromptTemplate  # Langchain 프롬프트 템플릿 관련 모듈 프롬프트 템플릿
from langchain.llms import HuggingFacePipeline  # Langchain LLM 관련 모듈 HuggingFace 파이프라인을 사용하는 LLM

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [7]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
ls

gdrive/  sample_data/


In [9]:
%cd /content/gdrive/MyDrive/LLMStudy/rag

/content/gdrive/MyDrive/LLMStudy/rag


# 1. 모델 불러오기

In [10]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [9]:
'''
modelName = "rssaem/llama3_ko_8b_rs1"
revision= "main"

config = AutoConfig.from_pretrained(modelName, revision=revision)
model = AutoModelForCausalLM.from_pretrained(modelName, revision=revision, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(modelName, revision=revision)
print(model)
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

# 2. 문서 불러오기

In [11]:
!pwd

/content/gdrive/MyDrive/LLMStudy/rag


In [12]:
ls /content/gdrive/MyDrive/LLMStudy/rag/pdfs/

namu.pdf


In [13]:
ls /content/gdrive/MyDrive/LLMStudy/rag/pdfs/namu.pdf

/content/gdrive/MyDrive/LLMStudy/rag/pdfs/namu.pdf


In [14]:
loader = PyPDFLoader('/content/gdrive/MyDrive/LLMStudy/rag/pdfs/namu.pdf')
documents = loader.load()

# 3. 문서 분리(조각)

In [15]:
output = []
# text 정제
for page in documents:
    text = page.page_content
    output.append(text)

doc_chunks = []
for line in output:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300, # 최대 청크 길이
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""], #  텍스트를 청크로 분할하는 데 사용되는 문자 목록
        chunk_overlap=200, # 인접한 청크 간에 중복되는 문자 수
    )
    chunks = text_splitter.split_text(line)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk, metadata={ "source": 'namu.pdf', "page": i}
        )
        doc_chunks.append(doc)

# 4. 인덱스 파이프라인 생성

In [16]:
# HuggingFaceEmbeddings 클래스를 사용하여 임베딩 모델 로드
embed_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-nli"  # 한국어 문장 임베딩 모델
)

<ipython-input-16-8f325991ab4f>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Chroma 데이터베이스 디렉토리 및 컬렉션 이름 설정
persist_directory = "./chromadb"
collection_name = "rscode_documents"

# 기존의 Chroma 인덱스를 로드하거나 없으면 새로 생성
try:
    index = Chroma(
        embedding_function=embed_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    # 컬렉션이 존재하면 문서 추가
    index.add_documents(doc_chunks)
except Exception as e:
    print(f"Error loading collection: {e}")
    # 컬렉션이 없으면 새로 생성하여 문서 추가
    index = Chroma.from_documents(
        documents=doc_chunks,
        embedding=embed_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )

# 변경된 내용을 디스크에 저장
index.persist()

# 검색기를 생성하여 유사도 검색 수행
retriever = index.as_retriever(search_kwargs={"k": 1})  # 검색 결과 상위 1개 반환

<ipython-input-17-294e7dcf8e68>:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  index = Chroma(
<ipython-input-17-294e7dcf8e68>:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  index.persist()


# 5. llm rag 파이프라인 생성

In [21]:
modelName = model_id
# 모델 설정을 로드하여 GenerationConfig 객체 생성
gen_cfg = GenerationConfig.from_pretrained(modelName)
# 생성 설정 세부 조정
gen_cfg.max_new_tokens = 1024  # 생성할 최대 토큰 수
gen_cfg.temperature = 0.0000001  # 텍스트 생성의 무작위성을 낮추는 온도 설정
gen_cfg.return_full_text = True  # 생성된 텍스트 전체 반환
gen_cfg.do_sample = True  # 샘플링을 통해 텍스트 생성
gen_cfg.repetition_penalty = 1.11  # 반복 페널티 설정

# HuggingFace의 파이프라인을 사용하여 텍스트 생성 파이프라인 생성
# model.cuda()
pipe = pipeline(
    task="text-generation",  # 파이프라인 작업 유형 설정
    model=model,  # 사전 훈련된 언어 모델
    tokenizer=tokenizer,  # 모델에 맞는 토크나이저
    generation_config=gen_cfg,  # 생성 설정 적용
    # device=0  # 사용할 디바이스 설정 (기본 GPU를 사용할 경우 0으로 설정)
)

# LangChain의 HuggingFacePipeline을 사용하여 LLM 객체 생성
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-21-d4f7766ffb37>:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [22]:
# LLAMA 2 LLM에 권장되는 프롬프트 스타일을 사용합니다.
prompt_template = """
[INST] <>
You are a wise assistant. Answer questions accurately in Korean
<>
{context}

Question: {question} [/INST]
"""

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# RetrievalQA 체인을 생성합니다.
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,  # 사용할 LLM을 지정합니다.
    chain_type="stuff",  # 체인의 타입을 지정합니다.
    retriever=retriever,  # 문서를 검색할 때 사용할 retriever를 지정합니다.
    chain_type_kwargs={"prompt": prompt},  # 체인 타입에 대한 추가 인자를 지정합니다.
)

In [23]:
# 질의 내용 설정
query = "방탄소년단(BTS)의 콘셉트는?"

# 질의에 대한 답변을 생성합니다.
result = Chain_pdf.invoke(query)
result

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{'query': '방탄소년단(BTS)의 콘셉트는?',
 'result': "\n[INST] <>\nYou are a wise assistant. Answer questions accurately in Korean\n<>\n한다. \n• 방탄소년단의  첫 대상 수상작이자  본격적으로  이름을  세계에  알리게  해준 정규 2\n집 앨범 《WINGS 》[33]로 넘어가면서는  성장한  음악성과  더불어  청춘이  성장하면서  \n겪는 고통과  유혹 그리고  성숙에  관해서  이야기한다 . 때때로  노래 가사에서  특정 \n지명이나  상표를  직접 언급하는  때도 있어 지상파  방송 불가 판정을  받기도  하지\n만 일부 앨범 수록곡에  해당하는  사항이다 .\n\nQuestion: 방탄소년단(BTS)의 콘셉트는? [/INST]\nAnswer: 방탄소년단(BTS)의 콘셉트는 성장, 성숙, 청춘, 고통, 유혹 등 다양한 주제를 다루며, 그들의 음악과 비디오, 라이브 공연 등을 통해 전달된다. 특히, 이들의 첫 번째 대중적인 성공을 기념하기 위해 'WINGS' 앨범은 청춘과 성장, 그리고 그 과정에서의 고통과 성숙을 다룬 내용이 많다. 또한, 노래 가사에서 특정 지명이나 상표를 언급할 때도 있다. 이들은 지상파 방송 불가 판정을 받기도 하며, 이러한 콘셉트는 그들의 음악과 예술적 표현에 큰 영향을 미친다."}